<a href="https://colab.research.google.com/github/azhgh22/Comparative-analysis-of-Generative-models-on-CIFAR-10/blob/main/experiments/train_sde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Set Env**

In [ ]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

from google.colab import userdata
token = userdata.get('GITHUB_TOKEN')
user_name = userdata.get('GITHUB_USERNAME')
mail = userdata.get('GITHUB_MAIL')

!git config --global user.name "{user_name}"
!git config --global user.email "{mail}"
!git clone https://{token}@github.com/azhgh22/Comparative-analysis-of-Generative-models-on-CIFAR-10.git

# **Imports**

In [ ]:
# Imports
import sys
import os
import torch
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from torchvision.utils import make_grid

# Add the root directory of the cloned repository to the Python path
sys.path.append('/content/Comparative-analysis-of-Generative-models-on-CIFAR-10')

import importlib
import data.cifar10 as cifar10_module
importlib.reload(cifar10_module)
from data.cifar10 import load_cifar10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


In [ ]:
train_loader, _ = load_cifar10(batch_size=128)

In [ ]:
import matplotlib.pyplot as plt

# Convert to HWC for plotting
def show_img(img):
  if img.dim() == 4:
      img = img[0]
  img = img.detach().cpu()
  img = img.clamp(0,1)
  img = img.permute(1,2,0)  # CHW -> HWC
  plt.figure(figsize=(4,4))
  plt.imshow(img, interpolation='nearest')
  plt.axis('off')
  plt.show()

In [ ]:
from models.scorebased_models.sde_diffusion import SDEDiffusion
from utils.get_device import get_device
from train.train import Train
from utils.checkpointer import Checkpointer

device = get_device()

model = SDEDiffusion(sde_type='VPSDE', device=device).to(device)

In [ ]:
print(device)

In [ ]:
checkpoint_dir = "/content/drive/MyDrive/checkpoints_final/sde"
checkpointer = Checkpointer(checkpoint_dir, "sde", 1, False)
train = Train(model, 10, train_loader, checkpointer, device)
train.load_checkpoint()

In [ ]:
print(model.device)

In [ ]:
train.train()

In [ ]:
train_dataset = train_loader.dataset
img = train_dataset[0][0]
show_img(img)

In [ ]:
gen_img = model.sample(16)

In [ ]:
def show_images(images, title="Images", n_row=4):
    # Display a grid of images
    images = images.cpu()
    # Denormalize from [-1, 1] to [0, 1]
    # images = (images + 1) / 2
    images = torch.clamp(images, 0, 1)

    n = len(images)
    n_col = n_row
    n_row = (n + n_col - 1) // n_col

    fig, axes = plt.subplots(n_row, n_col, figsize=(n_col * 2, n_row * 2))
    axes = axes.flatten() if n > 1 else [axes]

    for i, ax in enumerate(axes):
        if i < n:
            img = images[i].permute(1, 2, 0).numpy()
            ax.imshow(img)
            ax.axis('off')
        else:
            ax.axis('off')

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

In [ ]:
show_images(gen_img)